## MNIST
In this tutorial we will create a convolutional neural network that classifies handwritten numbers.



### Import dependencies
Start by importing the dependencies we will need for the project

In [ ]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from uoa_mlaas import use_gpu
use_gpu()

In [ ]:
K.set_image_dim_ordering('th')

### Set seed
Set a seed value so that when we repeatedly run our code we will get the same result. Using the same seed is important when you want to compare algorithms.

In [ ]:
seed = 7
numpy.random.seed(seed)

### Import data

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
num_classes = y_test.shape[1]

### Create the model

In [ ]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

### Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit the model

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

### Evaluate the model

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Error: {0:.2f}%".format((100-scores[1]*100)))